Vamos testar algumas técnicas para tentar incrementar o resultado obtido no primeiro estudo. Para isso, vamos testar o desempenho de outros modelos  e tentar ajustar seus parâmetros. No TCC de Mayara, ela obteve um bom resultado utilizando Naive Bayes. Para ajuste de parâmetros, acho que vale a pena testar o parâmetro C do LinearSVC para valores um pouco mais altos, utilizar a parametrização L2 e testar também a regressão logística, também com valores altos para o parâmetro C.

Ao tentar melhorar o desempenho, creio que não faz sentido utilizar o parâmetro pre-punning, uma vez que o modelo não está sofrendo *overfitting*, penso que lidar apenas com o número de estimators seja suficiente.

O algoritmo *Random Forest* busca diminuir o *overfitting* da *Decision Tree* medindo o desempenho de várias árvores. Como nossos modelos não estão sofrendo disso, talvez um menor número de estimadores tenha um desempenho melhor. Vamos testar.

Acho que uma opção é verificar os atributos mais importantes, selecionar apenas esses e remover a aleatoriedade dos atributos no treinamento da *random forest*.


In [1]:
import pickle
import os
import spacy
import pandas as pd
import numpy as np
from spacy.cli.download import download
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

In [2]:
def fazer_amostragem(train_dataset: pd.DataFrame):
    """

    :param train_dataset:
    :return:
    """
    sentimentos = train_dataset['Sentimento'].unique()
    df = pd.DataFrame([])

    for sentimento in sentimentos:
        df_filtrado = train_dataset.loc[train_dataset['Sentimento'] == sentimento][:600]
        df = pd.concat([df, df_filtrado])

    return df


def ler_modelo(path: str):
    return pickle.load(open(path, 'rb'))


def salvar_modelo(path: str, modelo):
    return pickle.dump(modelo, open(path, 'wb'))    


def criar_embeddings(df_tweets):
    try:
        nlp = spacy.load('pt_core_news_lg')
    except (IOError, OSError):
        download('pt_core_news_lg')
        nlp = spacy.load('pt_core_news_lg')
    # desativamos todos os outros pipes que vem com o modelo nlp porque não preicsaremos deles
    with nlp.disable_pipes():
        # transformamos cada texto em um vetor e colocamos em uma array
        print('Fazendo os word embeddings')
        vetores = np.array([nlp(texto).vector for texto in df_tweets.Texto])

    return vetores

In [3]:
path_datasets = '../resources/datasets'

# importação dos dados
df_treinamento = pd.read_csv(
    f'{path_datasets}/tweets_ekman.csv',
    usecols=['Texto', 'Sentimento']
).dropna()
df_alvo = pd.read_csv(f'{path_datasets}/tweets_pandemia.csv')

x_trainval, x_test, y_trainval, y_test = train_test_split(
    df_treinamento,
    df_treinamento['Sentimento'],
    test_size=0.2,
    random_state=42
)

x_train, x_valid, y_train, y_valid = train_test_split(
    x_trainval,
    y_trainval,
    random_state=42
)

In [4]:
# processamento dos dados para word embeddings
path_embeddings_treinamento = '../resources/modelos/embeddings_treinamento_val.pkl'
path_embeddings_teste = '../resources/modelos/embeddings_teste_grid.pkl'

if os.path.exists(path_embeddings_treinamento):
    embeddings = ler_modelo(path_embeddings_treinamento)
else:
    embeddings = criar_embeddings(x_train)
    salvar_modelo(path_embeddings_treinamento, embeddings)

if os.path.exists(path_embeddings_teste):
    embeddings_teste = ler_modelo(path_embeddings_teste)
else:
    embeddings_teste = criar_embeddings(x_test)
    salvar_modelo(path_embeddings_teste, embeddings_teste)

In [5]:
logreg = LogisticRegression(random_state=0, n_jobs=-1)

In [6]:
embeddings = embeddings[:len(x_train)]
parametros = {
    'penalty': ['l1', 'l2'],
    'C': [1, 3, 5, 6, 7, 8, 9, 10, 15, 20, 50],
    'solver': ['newton-cg', 'sag', 'newton_cholesky', 'saga', 'libfgs']
}

logreg_grid = GridSearchCV(logreg, parametros, cv=5, n_jobs=-1)
logreg_grid.fit(embeddings, y_train)
previsoes_lgr_best = logreg_grid.predict(embeddings_teste)
print(classification_report(y_test, previsoes_lgr_best))
print(logreg_grid.best_params_)

D:\home\Desenvolvimento\Analise-de-sentimentos-pandemia-covid19\emocoes_venv\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
330 fits failed out of a total of 550.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
55 fits failed with the following error:
Traceback (most recent call last):
  File "D:\home\Desenvolvimento\Analise-de-sentimentos-pandemia-covid19\emocoes_venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "D:\home\Desenvolvimento\Analise-de-sentimentos-pandemia-covid19\emocoes_venv\Lib\site-packages\sklearn\base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
           

              precision    recall  f1-score   support

      Neutro       0.96      0.95      0.96       495
       feliz       0.71      0.72      0.71      1931
        medo       0.54      0.61      0.57      2320
        nojo       0.60      0.60      0.60      1648
       raiva       0.41      0.12      0.18       562
      triste       0.52      0.54      0.53      2097

    accuracy                           0.60      9053
   macro avg       0.62      0.59      0.59      9053
weighted avg       0.60      0.60      0.59      9053

{'C': 5, 'penalty': 'l2', 'solver': 'sag'}


In [7]:
salvar_modelo(path='../resources/modelos/logreg_otimizado.pkl', modelo=logreg_grid.best_estimator_)